In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

#### Table 1: Tidy Processed Cleveland Dataset

In [ ]:
cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
        col_names = FALSE)

colnames(cleveland_data) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

tidy_cleveland_data <- cleveland_data |>
select("age", "trestbps", "chol", "thalach", "oldpeak", "thal") |>
mutate(across(age:thal, ~na_if(., "?"))) #This line of code was referenced from Wickham et al. (n.d.)
tidy_cleveland_data$thal[tidy_cleveland_data$thal == "3.0"] <- "normal" #This line of code was referenced from Naveen (2022)
tidy_cleveland_data$thal[tidy_cleveland_data$thal == "6.0"] <- "fixed_defect" #This line of code was referenced from Naveen (2022)
tidy_cleveland_data$thal[tidy_cleveland_data$thal == "7.0"] <- "reversible_defect" #This line of code was referenced from Naveen (2022)
tidy_cleveland_data <- mutate(tidy_cleveland_data, thal = as.factor(thal))
tidy_cleveland_data

#### Table 2: Training Cleveland Dataset

In [ ]:
set.seed(234) 
cleveland_split <- initial_split(tidy_cleveland_data, prop = 0.75, strata = thal)  
cleveland_train <- training(cleveland_split)   
cleveland_train

#### Table 3: Testing Cleveland Dataset

In [ ]:
set.seed(234) 
cleveland_test <- testing(cleveland_split)
cleveland_test

#### Table 4: Means of the Predictor Variables 

In [ ]:
cleveland_table <- cleveland_train |>
summarize(across(age:oldpeak, mean))
cleveland_table

#### Table 5: Number of Observations in Each Class

In [ ]:
cleveland_table_2 <- group_by(cleveland_train, thal) |>
summarize(count = n())
cleveland_table_2

#### Table 6: Rows That Have Missing Data

In [ ]:
cleveland_table_3 <- cleveland_train[!complete.cases(cleveland_train), ] #This line of code was referenced from Gili (2022)
cleveland_table_3

#### Table 7: Number of Rows That Have Missing Data per Column

In [ ]:
cleveland_table_4 <- cleveland_train |>
    map_df(~ sum(is.na(.))) #This line of code was referenced from Kiniry (2019).
cleveland_table_4

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)
cleveland_plot_1 <- cleveland_train |>
    ggplot(aes(x = trestbps, fill = thal)) +
    geom_histogram(binwidth = 5) +
    labs(x = "Resting Blood Pressure on Admission to Hospital (mmHg)", y = "Number of Patients",  fill = "Thallium Scintigraphy Test Results") +
    ggtitle("Figure 1: The Resting Blood Pressure of Patients on Admission to Hospital") +
    scale_fill_brewer(palette = "Reds")+
    theme(text = element_text(size = 20))
cleveland_plot_1

In [ ]:
cleveland_plot_2 <- cleveland_train |>
    ggplot(aes(x = age, fill = thal)) +
    geom_histogram(binwidth = 2) +
    labs(x = "Age (Years)", y = "Number of Patients",  fill = "Thallium Scintigraphy Test Results") +
    ggtitle("Figure 2: The Age of Patients") +
    scale_fill_brewer(palette = "Reds") +
    theme(text = element_text(size = 20))
cleveland_plot_2

In [ ]:
cleveland_plot_3 <- cleveland_train |>
    ggplot(aes(x = chol, fill = thal)) +
    geom_histogram(binwidth = 10) +
    labs(x = "Serum Cholesterol (mg/dl)", y = "Number of Patients",  fill = "Thallium Scintigraphy Test Results") +
    ggtitle("Figure 3: The Serum Cholesterol Levels of Patients") +
    scale_fill_brewer(palette = "Reds") +
    theme(text = element_text(size = 20))
cleveland_plot_3

In [ ]:
cleveland_plot_4 <- cleveland_train |>
    ggplot(aes(x = thalach, fill = thal)) +
    geom_histogram(binwidth = 5) +
    labs(x = "Maximum Heart Rate Achieved (bpm)", y = "Number of Patients",  fill = "Thallium Scintigraphy Test Results") +
    ggtitle("Figure 4: The Maximum Heart Rate Achieved by Patients") +
    scale_fill_brewer(palette = "Reds") +
    theme(text = element_text(size = 20))
cleveland_plot_4

In [ ]:
cleveland_plot_5 <- cleveland_train |>
    ggplot(aes(x = oldpeak, fill = thal)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "ST Depression Induced by Exercise Relative to Rest (Units)", y = "Number of Patients",  fill = "Thallium Scintigraphy Test Results") +
    ggtitle("Figure 5: The Value of the ST Depression Induced by Exercise Relative to Rest in Patients") +
    scale_fill_brewer(palette = "Reds") +
    theme(text = element_text(size = 20))
cleveland_plot_5

In [ ]:
set.seed(69)

cleveland_recipe <- recipe(thal ~ ., data = cleveland_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
       set_engine("kknn") |>
       set_mode("classification")

cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = thal)

k_value <- tibble(neighbors = seq(from = 2, to = 20, by = 1))

cleveland_results <- workflow() |>
       add_recipe(cleveland_recipe) |>
       add_model(knn_tune) |>
       tune_grid(resamples = cleveland_vfold, grid = k_value) |>
       collect_metrics()

cross_val_plot <- cleveland_results |>
filter(.metric == "accuracy") |>
ggplot(aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors (k)", y = "Accuracy")
cross_val_plot


In [ ]:
cleveland_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 17) |>
       set_engine("kknn") |>
       set_mode("classification")

cleveland_fit <- workflow() |>
       add_recipe(cleveland_recipe) |>
       add_model(cleveland_spec) |>
       fit(data = cleveland_train)
cleveland_fit


In [ ]:
cleveland_predictions <- predict(cleveland_fit, cleveland_test) |>
       bind_cols(cleveland_test)
cleveland_predictions

cleveland_metrics <- cleveland_predictions |>
metrics(truth = thal, estimate = .pred_class) |>
filter(.metric == "accuracy")
cleveland_metrics

cleveland_conf_mat <- cleveland_predictions |>
       conf_mat(truth = thal, estimate = .pred_class)
cleveland_conf_mat

In [ ]:
cleveland_misclass <- cleveland_predictions |>
                filter(.pred_class != thal)

misclass_plot_1_pred <- ggplot(cleveland_misclass, aes(x = age, y = trestbps, color = .pred_class)) +
                geom_point() +
                labs(x = "Age (years)", y = "Resting Blood Pressure on Admission to Hospital (mmHg)", 
                     title = "Figure 1: Comparing the Effects of Age and Resting Blood Pressure on Classification", 
                     color = "Thallium Scintigraphy Test Results") +
                theme(text = element_text(size = 20))

misclass_plot_1_pred


In [ ]:
misclass_plot_2_pred <- ggplot(cleveland_misclass, aes(x = chol, y = thalach, color = .pred_class)) +
                geom_point() +
                labs(x = "Serum Cholesterol (mg/dl)", y = "Maximum Heart Rate Achieved (bpm)", 
                     title = "Figure 2: Comparing the Effects of Cholesterol and Maximum Heart Rate on Classification", 
                     color = "Thallium Scintigraphy Test Results") +
                theme(text = element_text(size = 20))
misclass_plot_2_pred


In [ ]:
misclass_plot_3_pred <- ggplot(cleveland_misclass, aes(x = age, y = oldpeak, color = .pred_class)) +
                geom_point() +
                labs(x = "Age (years)", y = "ST Depression Induced by Exercise Relative to Rest (Units)", 
                     title = "Figure 3: Comparing the Effects of Age and ST Depression on Classification", 
                     color = "Thallium Scintigraphy Test Results") +
                theme(text = element_text(size = 20))
misclass_plot_3_pred